# ClimateWins Weather Prediction : Keras Layered Model (RNN/LSTM)

## This script contains:
### 1. Import Libraries and Data
### 2. Data Wrangling
### 3. Split the Data
### 4. Keras Model 
### 5. Compiling and Running Model
### 6. Confusion Matrix 
### 7. Keras Model Retrials (Until Convergence)

## 1. Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Set display options to show all columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Create a path for faster data importation
path = r'C:\Users\supri\Desktop\Supriya\DataAnalysis\Project\Data Immersion\MachineLearningWithPython\ClimateWins\Data Sets'

In [4]:
X = pd.read_csv(os.path.join(path, 'weather_cleaned.csv'))
prediction = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
X.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.018,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,8,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.018,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.01,

In [5]:
X.shape

(22950, 135)

In [6]:
prediction.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
prediction.shape

(22950, 16)

## 2. Data Wrangling

In [8]:
# Drop DATE column from answers

prediction.drop(columns = 'DATE', inplace = True)

In [9]:
prediction.shape

(22950, 15)

In [10]:
y = prediction

In [11]:
X.shape

(22950, 135)

In [12]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [13]:
X = X.reshape(-1,15,9)

In [14]:
X.shape

(22950, 15, 9)

In [15]:
y.shape

(22950, 15)

In [16]:
X

array([[[  7.    ,   0.85  ,   1.018 , ...,   6.5   ,   0.8   ,
          10.9   ],
        [  1.    ,   0.81  ,   1.0195, ...,   3.7   ,  -0.9   ,
           7.9   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.4   ,  -0.4   ,
           5.1   ],
        ...,
        [  4.    ,   0.73  ,   1.0304, ...,  -5.9   ,  -8.5   ,
          -3.2   ],
        [  5.    ,   0.98  ,   1.0114, ...,   4.2   ,   2.2   ,
           4.9   ],
        [  5.    ,   0.88  ,   1.0003, ...,   8.5   ,   6.    ,
          10.9   ]],

       [[  6.    ,   0.84  ,   1.018 , ...,   6.1   ,   3.3   ,
          10.1   ],
        [  6.    ,   0.84  ,   1.0172, ...,   2.9   ,   2.2   ,
           4.4   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.3   ,   1.4   ,
           3.1   ],
        ...,
        [  6.    ,   0.97  ,   1.0292, ...,  -9.5   , -10.5   ,
          -8.5   ],
        [  5.    ,   0.62  ,   1.0114, ...,   4.    ,   3.    ,
           5.    ],
        [  7.    ,   0.91  ,   1.0007, ...,   8.

## 3.  Split the data

In [17]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [18]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


## 4.  Keras Model 

In [19]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh')) # Don't use relu here!

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,215 (137.56 KB)

 Trainable params: 35,215 (137.56 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Compiling and Running Model

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 10s - 9ms/step - accuracy: 0.4023 - loss: 24.8599
Epoch 2/30
1076/1076 - 5s - 5ms/step - accuracy: 0.3226 - loss: 24.7977
Epoch 3/30
1076/1076 - 6s - 5ms/step - accuracy: 0.0916 - loss: 24.9287
Epoch 4/30
1076/1076 - 5s - 4ms/step - accuracy: 0.0447 - loss: 25.1899
Epoch 5/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0623 - loss: 24.7701
Epoch 6/30
1076/1076 - 6s - 5ms/step - accuracy: 0.0512 - loss: 24.7413
Epoch 7/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0404 - loss: 24.5039
Epoch 8/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0503 - loss: 24.6437
Epoch 9/30
1076/1076 - 6s - 6ms/step - accuracy: 0.0683 - loss: 24.7599
Epoch 10/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1386 - loss: 24.6704
Epoch 11/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1792 - loss: 24.6598
Epoch 12/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1846 - loss: 24.6208
Epoch 13/30
1076/1076 - 6s - 5ms/step - accuracy: 0.2187 - loss: 24.8215
Epoch 14/30
1076/1076 - 6s - 6ms/step - accuracy: 0.2662 - 

### 6. Confusion Matrix 

In [23]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [24]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [25]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Pred        MUNCHENB
True                
BASEL           3682
BELGRADE        1092
BUDAPEST         214
DEBILT            82
DUSSELDORF        29
HEATHROW          82
KASSEL            11
LJUBLJANA         61
MAASTRICHT         9
MADRID           458
MUNCHENB           8
OSLO               5
STOCKHOLM          4
VALENTIA           1


### 7. Keras Model Retrials (Until Convergence)

In [26]:
epochs = 30
batch_size = 16
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [27]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1475 - loss: 811.0411
Epoch 2/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1360 - loss: 7782.2617
Epoch 3/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1346 - loss: 24257.5293
Epoch 4/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1327 - loss: 55355.9648
Epoch 5/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1304 - loss: 100862.6797
Epoch 6/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1283 - loss: 155393.3750
Epoch 7/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1318 - loss: 226505.2500
Epoch 8/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1331 - loss: 315587.7188
Epoch 9/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1305 - loss: 418401.4062
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1341 - loss: 543758.1875
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1292 - loss: 686225.7500
Epoch 12/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1276 - loss: 847047.4375
Epoch 13/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1270 - loss: 1040748.1875
Epoch 14/30
10

In [30]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [31]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  HEATHROW  KASSEL  LJUBLJANA  \
True                                                                         
BASEL         572        74         2      36       256     236         43   
BELGRADE      369        27         0       7        45      38          3   
BUDAPEST       45         3         0       2        22       7          0   
DEBILT         25         0         0       1        10       6          0   
DUSSELDORF      3         0         0       1         3       2          0   
HEATHROW        4         1         0       2        14       3          0   
KASSEL          4         0         0       1         0       1          0   
LJUBLJANA       6         0         0       1         5       0          0   
MAASTRICHT      1         0         0       0         0       2          0   
MADRID         11         0         0       8        45      11          1   
MUNCHENB        3      

### Softmax is not producing good results, try with tanh, sigmoid, and Relu.

In [32]:
epochs = 30
batch_size = 16
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [33]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14, 16)         │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,191 (24.18 KB)

 Trainable params: 6,191 (24.18 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [84]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 1s - 883us/step - accuracy: 0.0033 - loss: 25.4100
Epoch 2/30
1076/1076 - 1s - 573us/step - accuracy: 0.0012 - loss: 25.7457
Epoch 3/30
1076/1076 - 1s - 580us/step - accuracy: 9.2958e-04 - loss: 25.7429
Epoch 4/30
1076/1076 - 1s - 585us/step - accuracy: 9.2958e-04 - loss: 25.7429
Epoch 5/30
1076/1076 - 1s - 570us/step - accuracy: 9.8768e-04 - loss: 25.7391
Epoch 6/30
1076/1076 - 1s - 571us/step - accuracy: 9.8768e-04 - loss: 25.7391
Epoch 7/30
1076/1076 - 1s - 652us/step - accuracy: 9.8768e-04 - loss: 25.7372
Epoch 8/30
1076/1076 - 1s - 658us/step - accuracy: 9.8768e-04 - loss: 25.7363
Epoch 9/30
1076/1076 - 1s - 580us/step - accuracy: 9.8768e-04 - loss: 25.7363
Epoch 10/30
1076/1076 - 1s - 584us/step - accuracy: 9.8768e-04 - loss: 25.7363
Epoch 11/30
1076/1076 - 1s - 603us/step - accuracy: 9.8768e-04 - loss: 25.7363
Epoch 12/30
1076/1076 - 1s - 589us/step - accuracy: 9.8768e-04 - loss: 25.7363
Epoch 13/30
1076/1076 - 1s - 610us/step - accuracy: 9.8768e-04 - loss

In [34]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])
    

In [35]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BUDAPEST  DUSSELDORF  KASSEL  MADRID  MUNCHENB  OSLO  \
True                                                                      
BASEL          42       118         162     172      17      3169     2   
BELGRADE        0         0          33       0       0      1059     0   
BUDAPEST        0         0           6       0       0       207     0   
DEBILT          0         0           0       0       0        82     0   
DUSSELDORF      0         0           1       0       0        28     0   
HEATHROW        0         0           2       0       0        80     0   
KASSEL          0         0           0       0       0        11     0   
LJUBLJANA       0         0           5       0       0        56     0   
MAASTRICHT      0         0           0       0       0         9     0   
MADRID          0        17          24       1       0       416     0   
MUNCHENB        0         0           0       0       0    

### Model's loss is decreasing, but improvement not reflected in accuracy.Try adjusting layers.

In [36]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [37]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0985 - loss: 26.8559
Epoch 2/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1132 - loss: 27.7468
Epoch 3/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1134 - loss: 27.7478
Epoch 4/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1134 - loss: 27.7478
Epoch 5/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 6/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 7/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7496
Epoch 8/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 9/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 10/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 11/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 12/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1135 - loss: 27.7496
Epoch 13/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - loss: 27.7497
Epoch 14/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1135 - l

In [40]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [41]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BELGRADE  BUDAPEST  DUSSELDORF  HEATHROW  KASSEL  LJUBLJANA  OSLO
True                                                                         
BASEL           1366       600           1       110       5        499  1101
BELGRADE         609        94           0        44       0        344     1
BUDAPEST         119        16           0        14       0         63     2
DEBILT            66         3           0         5       0          7     1
DUSSELDORF        22         2           0         0       0          5     0
HEATHROW          41         5           0         7       0         24     5
KASSEL             8         0           0         0       0          3     0
LJUBLJANA         16        14           0         3       0         22     6
MAASTRICHT         2         1           0         0       0          6     0
MADRID           119        78           0         9       0        153    99
MUNCHENB           5   

### Loss is not is stable but not as low as expected, accuracy is much higher. Try with sigmoid and relu.

In [42]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [43]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 6s - 5ms/step - accuracy: 0.6195 - loss: 7857.0068
Epoch 2/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6435 - loss: 83250.7266
Epoch 3/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6437 - loss: 290577.3750
Epoch 4/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 626816.6875
Epoch 5/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 1141519.7500
Epoch 6/30
1076/1076 - 5s - 4ms/step - accuracy: 0.6439 - loss: 1799058.0000
Epoch 7/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6439 - loss: 2628929.7500
Epoch 8/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 3660420.7500
Epoch 9/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 4942518.0000
Epoch 10/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 6406508.0000
Epoch 11/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 8170651.0000
Epoch 12/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 10050315.0000
Epoch 13/30
1076/1076 - 3s - 3ms/step - accuracy: 0.6440 - loss: 12325321.0000


In [46]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [47]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1
